In [42]:
#pip install catboost

In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import os

pd.options.display.max_columns = None

In [44]:
import warnings
warnings.filterwarnings("ignore")

In [45]:
# Путь к директории с данными

data_dir = r'e:\Documents\mchs\ice'
os.listdir(data_dir)

['hydro_1day.csv',
 'hydro_coord.csv',
 'ice_saw.csv',
 'meteo_1day.csv',
 'meteo_1month.csv',
 'meteo_3hours.csv',
 'meteo_coord.csv',
 'reference_horiz_visib.csv',
 'reference_water_codes.csv',
 'test.csv',
 'train.csv']

In [46]:
# Загрузим всё
main_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))

# Метео
mc = pd.read_csv(os.path.join(data_dir, 'meteo_coord.csv'))
mld = pd.read_csv(os.path.join(data_dir, 'meteo_1day.csv'))
mlm = pd.read_csv(os.path.join(data_dir, 'meteo_1month.csv'))
rhv = pd.read_csv(os.path.join(data_dir, 'reference_horiz_visib.csv'))

# Гидро
hc = pd.read_csv(os.path.join(data_dir, 'hydro_coord.csv'))
hld = pd.read_csv(os.path.join(data_dir, 'hydro_1day.csv'),
                   parse_dates=['date'])
rwc = pd.read_csv(os.path.join(data_dir, 'reference_water_codes.csv'))

In [125]:
# TEST
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))

In [47]:
# Подтянем ближайшую к гидростанции метеостанцию

from geopy.distance import geodesic
import re

def merge_coord(df):
    df['lat_long'] = df[['lat', 'lon']].apply(tuple, axis=1)
    return df

def stat_km(point, stat_list):
    stations_list=stat_list
    lst=[]
    if pd.isnull(point):
        lst.append(np.nan)
    else:
        for i in stations_list['lat_long']:
            x=geodesic(point, i).km
            lst.append(x)
            stations_list['dist']=pd.DataFrame(lst)
        y=stations_list['station_id'][stations_list['dist'] == stations_list['dist'].min()]
        y=y.to_string()
        y=re.sub("^[0-9]+", "", y)
        y=re.sub(" +", "", y)
        return int(y)

hc = merge_coord(hc)
mc = merge_coord(mc)
hc['closest_hydro'] = mc.lat_long.apply(lambda x: stat_km(x, mc))

In [48]:
mld = mld.merge(hc[['station_id', 'closest_hydro']], left_on=['station_id'], 
                right_on='closest_hydro', how='right')
mld.drop(['station_id_x', 'closest_hydro', 'date'], axis=1, inplace=True)
mld.rename({'station_id_y': 'station_id'}, axis=1, inplace=True)

### Почистим пропуски

#### mld

In [49]:
mld.shape

(237776, 46)

In [50]:
mld.isna().sum()

year                               0
month                              0
day                                0
route_type                    232756
snow_coverage_near_station    236241
snow_coverage_route           232758
ice_crust_route               237769
snow_height_aver              232760
snow_height_max               232760
snow_height_min               232795
snow_density_aver             233156
ice_crust_aver                237769
snow_saturated_thickness      237604
water_thickness               236200
water_in_snow                 233117
water_total                   233029
snow_coverage_charact         232969
snow_charact                  234463
snow_height                       14
snow_coverage_station             14
snow_height_q1                    14
snow_height_q2                    14
snow_height_q3                    14
temperature_20cm              142544
temperature_20cm_qual         142544
temperature_40cm              142544
temperature_40cm_qual         142544
t

In [51]:
col_save = mld.isna().sum()[mld.isna().sum() == 14].index

In [52]:
mld[col_save] = mld[col_save].fillna(method='ffill')
mld.dropna(axis='columns', inplace=True)

In [53]:
mld.isna().sum()

year                     0
month                    0
day                      0
snow_height              0
snow_coverage_station    0
snow_height_q1           0
snow_height_q2           0
snow_height_q3           0
station_id               0
dtype: int64

#### mlm

In [54]:
mlm.shape

(10060, 12)

In [55]:
mlm.isna().sum()

station_id                           0
year                                 0
month                                0
data_qual                         1782
precipitation_observed            1782
precipitation_corrected           1782
precipitation_corrected_liquid    1782
precipitation_corrected_mixed     1782
precipitation_corrected_solid     1782
sunshine_hours                    3013
date                                 0
day                                  0
dtype: int64

Удалим столбцы `data_qual` - на мой взгляд не несет критической информации для задачи, `sunshine_hours` - т.к. почти треть значений пропуски.

In [56]:
mlm = mlm.drop(columns=['data_qual', 'sunshine_hours'])

In [57]:
empty_columns = mlm.isna().sum()[mlm.isna().sum() > 0].index

In [58]:
#mlm[empty_columns]

Необходимо заменить категориальные переменные 9999.9 на NaN и далее заполнить forwardfill.

In [59]:
#mlm[empty_columns].describe()

In [60]:
mlm[empty_columns] = mlm[empty_columns].fillna(method='ffill')

In [61]:
a = mlm[empty_columns].max().max()

In [62]:
mlm[empty_columns] = mlm[empty_columns].replace(a, np.nan).fillna(method='ffill')

In [63]:
mlm.isna().sum()

station_id                        0
year                              0
month                             0
precipitation_observed            0
precipitation_corrected           0
precipitation_corrected_liquid    0
precipitation_corrected_mixed     0
precipitation_corrected_solid     0
date                              0
day                               0
dtype: int64

#### hc

In [64]:
hc.isna().sum()

station_id              0
name                    0
lat                     0
lon                     0
distance_from_source    0
drainage_area           0
z_null                  0
lat_long                0
closest_hydro           0
dtype: int64

#### hld

In [65]:
hld.shape

(224848, 14)

In [66]:
hld.isna().sum()

year                  0
station_id            0
month                 0
day                   0
date                  0
stage_avg            95
stage_min            95
stage_max            95
temp              76153
water_code          110
ice_thickness    216119
snow_height      216205
place            216020
discharge        139686
dtype: int64

In [67]:
drop_col = hld.isna().sum()[hld.isna().sum() > (hld.shape[0]) * .3].index

In [68]:
hld = hld.drop(columns=drop_col)

In [69]:
hld = hld.fillna(method='ffill')

In [70]:
hld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224848 entries, 0 to 224847
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   year        224848 non-null  int64         
 1   station_id  224848 non-null  int64         
 2   month       224848 non-null  int64         
 3   day         224848 non-null  int64         
 4   date        224848 non-null  datetime64[ns]
 5   stage_avg   224848 non-null  float64       
 6   stage_min   224848 non-null  float64       
 7   stage_max   224848 non-null  float64       
 8   water_code  224848 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 15.4+ MB


In [71]:
hld.isna().sum()

year          0
station_id    0
month         0
day           0
date          0
stage_avg     0
stage_min     0
stage_max     0
water_code    0
dtype: int64

In [72]:
#rwc

### Разделим годы на train и test сбалансированно по суммарной продолжительности заторных событий

In [73]:
# Суммарное количество заторов в году
jams_by_year = main_df.groupby('year').sum()['ice_jam'].to_frame().reset_index()
jams_by_year['ice_jam'].describe()

count    26.000000
mean      5.307692
std       3.792300
min       0.000000
25%       3.000000
50%       5.000000
75%       8.000000
max      16.000000
Name: ice_jam, dtype: float64

In [74]:
# Разделим на бины по квартилям
bins = [-1, 3, 5, 8, 16]
jams_by_year['ice_jam_bins'] = pd.cut(jams_by_year['ice_jam'], bins)
X_length = jams_by_year[['year', 'ice_jam']]
y_length = jams_by_year['ice_jam_bins']
y_length.value_counts()

(-1, 3]    9
(3, 5]     7
(5, 8]     6
(8, 16]    4
Name: ice_jam_bins, dtype: int64

In [75]:
# Разделим годы на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X_length, 
                                                    y_length, 
                                                    test_size=.3,  
                                                    stratify=y_length, 
                                                    random_state=42)

In [76]:
test_task_1_random = pd.read_csv(r'e:\Documents\mchs\test_task_1_random.csv')
#(test_task_1_random['year'] - 1).unique()

In [77]:
X_test_col = list((test_task_1_random['year'] - 1).unique())
X_train_col = list(set(X_length.year.unique()) - set(X_test_col))

### Соберем фичи из гидроданных

<div class="alert alert-block alert-warning">
Мы не можем использовать данные из будущего: всё, что происходит после заторного периода, относится уже к следующему году.
</div>

In [78]:
# Внесем не темпоральные данные
main_df = pd.merge(main_df, hc[['station_id', 
                                'distance_from_source', 
                                'drainage_area', 
                                'z_null']], on='station_id', how='left')

# Возьмем также данные из ежедневных наблюдений
#hld = pd.read_csv(os.path.join(data_dir, 'hydro_1day.csv'),
#                   parse_dates=['date'])
hld.head(1)

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,water_code
0,2000,3019,1,1,2000-01-01,74.0,74.0,74.0,46


In [79]:
main_df

,year,station_id,day,ice_jam,distance_from_source,drainage_area,z_null
0,2000,3019,1,0.0,1140.0,92200,249.38
1,2000,3019,2,0.0,1140.0,92200,249.38
2,2000,3019,3,0.0,1140.0,92200,249.38
3,2000,3019,4,0.0,1140.0,92200,249.38
4,2000,3019,5,0.0,1140.0,92200,249.38
...,...,...,...,...,...,...,...
11272,1999,3027,39,0.0,1575.0,199000,175.51
11273,1999,3027,40,0.0,1575.0,199000,175.51
11274,1999,3027,41,0.0,1575.0,199000,175.51
11275,1999,3027,42,0.0,1575.0,199000,175.51


In [80]:
hld['station_id'] = hld.station_id.astype(int)
mld['station_id'] = mld['station_id'].astype(int)

In [81]:
hld.head()

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,water_code
0,2000,3019,1,1,2000-01-01,74.0,74.0,74.0,46
1,2000,3019,1,2,2000-01-02,70.0,70.0,70.0,46
2,2000,3019,1,3,2000-01-03,67.0,67.0,67.0,46
3,2000,3019,1,4,2000-01-04,64.0,64.0,64.0,46
4,2000,3019,1,5,2000-01-05,60.0,60.0,60.0,46


In [82]:
mld.head()

,year,month,day,snow_height,snow_coverage_station,snow_height_q1,snow_height_q2,snow_height_q3,station_id
0,1985,1,1,32.0,10.0,0.0,0.0,0.0,3019
1,1985,1,2,32.0,10.0,0.0,0.0,0.0,3019
2,1985,1,3,32.0,10.0,0.0,0.0,0.0,3019
3,1985,1,4,32.0,10.0,0.0,0.0,0.0,3019
4,1985,1,5,32.0,10.0,0.0,0.0,0.0,3019


In [83]:
hld = hld.merge(mld, on=['station_id', 'year', 'month', 'day'], how='inner')
#hld = hld.fillna(0)
hld.head()

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,water_code,snow_height,snow_coverage_station,snow_height_q1,snow_height_q2,snow_height_q3
0,2000,3019,1,1,2000-01-01,74.0,74.0,74.0,46,44.0,10.0,0.0,0.0,0.0
1,2000,3019,1,2,2000-01-02,70.0,70.0,70.0,46,44.0,10.0,0.0,0.0,0.0
2,2000,3019,1,3,2000-01-03,67.0,67.0,67.0,46,45.0,10.0,0.0,0.0,0.0
3,2000,3019,1,4,2000-01-04,64.0,64.0,64.0,46,45.0,10.0,0.0,0.0,0.0
4,2000,3019,1,5,2000-01-05,60.0,60.0,60.0,46,46.0,10.0,0.0,0.0,0.0


### Скорректируем год, в который доступно наблюдение

In [84]:
# Определим для наблюдения год, в который оно доступно

# Маска-окно между заторным периодом и концом года
def after_jam_window(row, local=False):
    if local:
        month = row.month_local
        day = row.date_local.day
    else:
        month = row.month
        day = row.date.day
    return (((month == 6) and (day > 3))
            or (month in [7, 8, 9, 10, 11, 12]))

# Год относительно бизнес-логики
def target_year(row, local=False):
    if local:
        year = row.year_local
    else:
        year = row.year
    if after_jam_window(row):
        return year + 1
    else:
        return year
    
hld['target_year'] = hld.apply(target_year, axis=1)

# Календарный год и день больше не нужны
hld.drop(columns=['year', 'date', 'day'], axis=1, inplace=True)

In [85]:
#rwc

In [86]:
water_code_dict = dict(enumerate(list(rwc['phase'].value_counts().index)))
inverted_dict = {v: k for k, v in water_code_dict.items()}
#inverted_dict

In [87]:
rwc['new_code'] = rwc['phase'].apply(lambda x: inverted_dict[x])

In [88]:
new_water_code_dict = dict(rwc[['water_code','new_code']].values)
#new_water_code_dict

In [89]:
hld['water_code'] = hld['water_code'].apply(lambda x: re.match("^[0-9]+", x).group())
hld['water_code'] = hld['water_code'].astype('uint16')
hld['water_code'] = hld['water_code'].apply(lambda x: new_water_code_dict[x] if x in new_water_code_dict else 0)
hld['water_code'] = hld['water_code'].astype('category')

In [90]:
hld['snow_coverage_station'].value_counts(normalize=True)

10.0    0.475732
0.0     0.308674
99.0    0.203903
8.0     0.002386
6.0     0.001656
9.0     0.001493
7.0     0.001402
4.0     0.001279
5.0     0.001007
2.0     0.000880
3.0     0.000844
1.0     0.000744
Name: snow_coverage_station, dtype: float64

20 % данных забраковано (код 99 говорит о том, что данные забракованы). Заполним также как ранее: на NaN и далее ffill

In [91]:
hld['snow_coverage_station'] = hld['snow_coverage_station'].replace(99.0, np.nan).fillna(method='ffill')

In [92]:
hld['snow_height'] = hld['snow_height'].replace(9999.0, np.nan).fillna(method='ffill')
hld['snow_height'].value_counts(normalize=True)

0.0      0.510813
28.0     0.012276
25.0     0.011877
27.0     0.011745
26.0     0.011727
           ...   
98.0     0.000009
103.0    0.000005
104.0    0.000005
109.0    0.000005
95.0     0.000005
Name: snow_height, Length: 104, dtype: float64

In [93]:
#hld.groupby(index).median()

In [94]:
main_df['ice_jam'] = main_df['ice_jam'].astype('uint8')
main_df['distance_from_source'] = main_df['distance_from_source'].astype('int32')
main_df[['station_id','ice_jam']] = main_df[['station_id','ice_jam']].astype('category')
main_df

,year,station_id,day,ice_jam,distance_from_source,drainage_area,z_null
0,2000,3019,1,0,1140,92200,249.38
1,2000,3019,2,0,1140,92200,249.38
2,2000,3019,3,0,1140,92200,249.38
3,2000,3019,4,0,1140,92200,249.38
4,2000,3019,5,0,1140,92200,249.38
...,...,...,...,...,...,...,...
11272,1999,3027,39,0,1575,199000,175.51
11273,1999,3027,40,0,1575,199000,175.51
11274,1999,3027,41,0,1575,199000,175.51
11275,1999,3027,42,0,1575,199000,175.51


In [95]:
hld[['snow_coverage_station','snow_height_q1','snow_height_q2','snow_height_q3']] = hld[['snow_coverage_station','snow_height_q1','snow_height_q2','snow_height_q3']].astype('category')

In [96]:
# Гидро – сделаем ресэмплинг по месяцу

index = ['station_id', 'month', 'target_year']

hld_median = hld.groupby(index).median().add_prefix('median_').reset_index()
hld_first = hld.groupby(index).first().add_prefix('first_').reset_index()
#hld_min = hld.groupby(index).first().add_prefix('first_').reset_index()
#hld_std = hld.groupby(index).std().add_prefix('std_').reset_index()
data_frames = [hld_median, hld_first]#, hld_first]#, hld_std]

hydro_monthly = pd.concat(data_frames, axis=1)
hydro_monthly = hydro_monthly.loc[:,~hydro_monthly.columns.duplicated()]
hydro_monthly.sort_values(index)

,station_id,month,target_year,median_stage_avg,median_stage_min,median_stage_max,median_snow_height,first_stage_avg,first_stage_min,first_stage_max,first_water_code,first_snow_height,first_snow_coverage_station,first_snow_height_q1,first_snow_height_q2,first_snow_height_q3
0,3019,1,1985,-26.0,-26.0,-26.0,33.0,-23.0,-23.0,-23.0,3,32.0,10.0,0.0,0.0,0.0
1,3019,1,1986,59.0,58.0,60.0,32.0,74.0,74.0,74.0,3,30.0,10.0,0.0,0.0,0.0
2,3019,1,1987,3.0,2.0,3.0,25.0,38.0,37.0,38.0,3,23.0,10.0,0.0,0.0,0.0
3,3019,1,1988,29.0,29.0,29.0,41.0,39.0,39.0,39.0,3,29.0,10.0,0.0,0.0,0.0
4,3019,1,1989,84.0,83.0,84.0,36.0,97.0,96.0,97.0,3,32.0,10.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,3555,12,2016,266.0,266.0,266.0,26.0,273.0,273.0,273.0,3,18.0,10.0,0.0,0.0,0.0
7746,3555,12,2017,292.0,292.0,292.0,36.0,301.0,301.0,301.0,3,26.0,10.0,0.0,0.0,0.0
7747,3555,12,2018,261.0,261.0,261.0,42.0,275.0,275.0,275.0,3,28.0,10.0,0.0,0.0,0.0
7748,3555,12,2019,295.0,295.0,295.0,48.0,284.0,284.0,284.0,3,43.0,10.0,0.0,0.0,0.0


In [97]:
def make_features(df):
    station, target_year = df.name
    result = pd.DataFrame()
    for month, mdf in df.groupby('month'):
        m_feats = mdf[df.columns[4:]].add_prefix(str(month) + '_').reset_index(drop=True)
        result = pd.concat([result, m_feats], axis=1)
    return result.reset_index(drop=True)
        
hydro_features = hydro_monthly.groupby(['station_id', 'target_year']).apply(make_features)
hydro_features = hydro_features.reset_index(level=2, drop=True).reset_index()
hydro_features.dropna(how='all', axis=1, inplace=True)
hydro_features

,station_id,target_year,1_median_stage_min,1_median_stage_max,1_median_snow_height,1_first_stage_avg,1_first_stage_min,1_first_stage_max,1_first_water_code,1_first_snow_height,1_first_snow_coverage_station,1_first_snow_height_q1,1_first_snow_height_q2,1_first_snow_height_q3,2_median_stage_min,2_median_stage_max,2_median_snow_height,2_first_stage_avg,2_first_stage_min,2_first_stage_max,2_first_water_code,2_first_snow_height,2_first_snow_coverage_station,2_first_snow_height_q1,2_first_snow_height_q2,2_first_snow_height_q3,6_median_stage_min,6_median_stage_max,6_median_snow_height,6_first_stage_avg,6_first_stage_min,6_first_stage_max,6_first_water_code,6_first_snow_height,6_first_snow_coverage_station,6_first_snow_height_q1,6_first_snow_height_q2,6_first_snow_height_q3,7_median_stage_min,7_median_stage_max,7_median_snow_height,7_first_stage_avg,7_first_stage_min,7_first_stage_max,7_first_water_code,7_first_snow_height,7_first_snow_coverage_station,7_first_snow_height_q1,7_first_snow_height_q2,7_first_snow_height_q3,8_median_stage_min,8_median_stage_max,8_median_snow_height,8_first_stage_avg,8_first_stage_min,8_first_stage_max,8_first_water_code,8_first_snow_height,8_first_snow_coverage_station,8_first_snow_height_q1,8_first_snow_height_q2,8_first_snow_height_q3,9_median_stage_min,9_median_stage_max,9_median_snow_height,9_first_stage_avg,9_first_stage_min,9_first_stage_max,9_first_water_code,9_first_snow_height,9_first_snow_coverage_station,9_first_snow_height_q1,9_first_snow_height_q2,9_first_snow_height_q3,10_median_stage_min,10_median_stage_max,10_median_snow_height,10_first_stage_avg,10_first_stage_min,10_first_stage_max,10_first_water_code,10_first_snow_height,10_first_snow_coverage_station,10_first_snow_height_q1,10_first_snow_height_q2,10_first_snow_height_q3,11_median_stage_min,11_median_stage_max,11_median_snow_height,11_first_stage_avg,11_first_stage_min,11_first_stage_max,11_first_water_code,11_first_snow_height,11_first_snow_coverage_station,11_first_snow_height_q1,11_first_snow_height_q2,11_first_snow_height_q3,12_median_stage_min,12_median_stage_max,12_median_snow_height,12_first_stage_avg,12_first_stage_min,12_first_stage_max,12_first_water_code,12_first_snow_height,12_first_snow_coverage_station,12_first_snow_height_q1,12_first_snow_height_q2,12_first_snow_height_q3
0,3019,1985,-26.0,-26.0,33.0,-23.0,-23.0,-23.0,3,32.0,10,0,0,0,-28.0,-28.0,37.0,-28.0,-28.0,-28.0,3,35.0,10,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3019,1986,58.0,60.0,32.0,74.0,74.0,74.0,3,30.0,10,0,0,0,26.0,27.0,37.0,36.0,36.0,36.0,3,35.0,10,0,0,0,325.0,329.0,0.0,336.0,331.0,341.0,0,0.0,10,1,0,0,66.0,71.0,0.0,193.0,191.0,195.0,0,0.0,10,1,0,0,13.0,14.0,0.0,-10.0,-10.0,-9.0,0,0.0,10,1,0,0,112.5,113.5,0.0,104.0,102.0,105.0,0,0.0,10,1,0,0,28.0,28.0,6.0,69.0,69.0,69.0,0,0.0,10,1,0,0,-37.0,-37.0,23.0,-14.0,-14.0,-14.0,2,16.0,10,0,0,0,104.0,104.0,26.0,85.0,85.0,85.0,3,25.0,10,0,0,0
2,3019,1987,2.0,3.0,25.0,38.0,37.0,38.0,3,23.0,10,0,0,0,-7.0,-7.0,26.0,-6.0,-6.0,-6.0,3,27.0,10,0,0,0,277.0,283.0,0.0,354.0,350.0,358.0,0,0.0,10,1,0,0,18.0,19.0,0.0,124.0,121.0,127.0,0,0.0,10,1,0,0,-17.0,-16.0,0.0,13.0,12.0,13.0,0,0.0,10,1,0,0,-5.0,-4.0,0.0,-11.0,-11.0,-10.0,0,0.0,10,1,0,0,-36.0,-33.0,0.0,-11.0,-11.0,-11.0,0,0.0,10,1,0,0,-43.0,-42.0,13.0,-106.0,-112.0,-99.0,3,8.0,10,0,0,0,38.0,41.0,22.0,14.0,13.0,14.0,3,18.0,10,0,0,0
3,3019,1988,29.0,29.0,41.0,39.0,39.0,39.0,3,29.0,10,0,0,0,14.0,14.5,41.0,19.0,19.0,19.0,3,41.0,10,0,0,0,261.0,273.0,0.0,355.0,346.0,363.0,0,0.0,10,1,0,0,111.0,114.0,0.0,143.0,143.0,143.0,0,0.0,10,1,0,0,-15.0,-14.0,0.0,29.0,27.0,30.0,0,0.0,10,1,0,0,28.0,28.5,0.0,-14.0,-14.0,-13.0,0,0.0,10,1,0,0,13.0,14.0,10.0,32.0,30.0,33.0,0,0.0,10,1,0,0,28.5,31.0,24.0,

### Соберем фичи в основной датасет

<div class="alert alert-block alert-success">
Важно: merge делаем по target_year.
</div>

In [98]:
main_df = pd.merge(main_df, hydro_features, left_on=['year', 'station_id'],
                   right_on=['target_year', 'station_id'],
                   how='left')
cols = main_df.columns.to_list()
main_df = main_df[cols[:3] + [cols[7]] + cols[5:7] + cols[8:] + [cols[3]]]
main_df.dropna(how='any',inplace=True)

In [99]:
#main_df = main_df.drop(columns='year')

In [100]:
main_df['station_id'] = main_df['station_id'].astype('category')

In [101]:
main_df

,year,station_id,day,target_year,drainage_area,z_null,1_median_stage_min,1_median_stage_max,1_median_snow_height,1_first_stage_avg,1_first_stage_min,1_first_stage_max,1_first_water_code,1_first_snow_height,1_first_snow_coverage_station,1_first_snow_height_q1,1_first_snow_height_q2,1_first_snow_height_q3,2_median_stage_min,2_median_stage_max,2_median_snow_height,2_first_stage_avg,2_first_stage_min,2_first_stage_max,2_first_water_code,2_first_snow_height,2_first_snow_coverage_station,2_first_snow_height_q1,2_first_snow_height_q2,2_first_snow_height_q3,6_median_stage_min,6_median_stage_max,6_median_snow_height,6_first_stage_avg,6_first_stage_min,6_first_stage_max,6_first_water_code,6_first_snow_height,6_first_snow_coverage_station,6_first_snow_height_q1,6_first_snow_height_q2,6_first_snow_height_q3,7_median_stage_min,7_median_stage_max,7_median_snow_height,7_first_stage_avg,7_first_stage_min,7_first_stage_max,7_first_water_code,7_first_snow_height,7_first_snow_coverage_station,7_first_snow_height_q1,7_first_snow_height_q2,7_first_snow_height_q3,8_median_stage_min,8_median_stage_max,8_median_snow_height,8_first_stage_avg,8_first_stage_min,8_first_stage_max,8_first_water_code,8_first_snow_height,8_first_snow_coverage_station,8_first_snow_height_q1,8_first_snow_height_q2,8_first_snow_height_q3,9_median_stage_min,9_median_stage_max,9_median_snow_height,9_first_stage_avg,9_first_stage_min,9_first_stage_max,9_first_water_code,9_first_snow_height,9_first_snow_coverage_station,9_first_snow_height_q1,9_first_snow_height_q2,9_first_snow_height_q3,10_median_stage_min,10_median_stage_max,10_median_snow_height,10_first_stage_avg,10_first_stage_min,10_first_stage_max,10_first_water_code,10_first_snow_height,10_first_snow_coverage_station,10_first_snow_height_q1,10_first_snow_height_q2,10_first_snow_height_q3,11_median_stage_min,11_median_stage_max,11_median_snow_height,11_first_stage_avg,11_first_stage_min,11_first_stage_max,11_first_water_code,11_first_snow_height,11_first_snow_coverage_station,11_first_snow_height_q1,11_first_snow_height_q2,11_first_snow_height_q3,12_median_stage_min,12_median_stage_max,12_median_snow_height,12_first_stage_avg,12_first_stage_min,12_first_stage_max,12_first_water_code,12_first_snow_height,12_first_snow_coverage_station,12_first_snow_height_q1,12_first_snow_height_q2,12_first_snow_height_q3,ice_jam
0,2000,3019,1,2000.0,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,3,44.0,10,0,0,0,24.5,24.5,49.0,27.0,27.0,27.0,3,48.0,10,0,0,0,160.0,165.0,0.0,198.0,192.0,204.0,0,0.0,10,1,0,0,65.0,68.0,0.0,73.0,71.0,75.0,0,0.0,10,1,0,0,82.0,88.0,0.0,89.0,81.0,96.0,0,0.0,10,1,0,0,72.5,76.0,0.0,107.0,106.0,107.0,0,0.0,10,1,0,0,12.0,14.0,19.0,38.0,38.0,38.0,0,1.0,10,0,0,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,0,0,82.0,82.0,37.0,62.0,62.0,62.0,3,32.0,10,0,0,0,0
1,2000,3019,2,2000.0,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,3,44.0,10,0,0,0,24.5,24.5,49.0,27.0,27.0,27.0,3,48.0,10,0,0,0,160.0,165.0,0.0,198.0,192.0,204.0,0,0.0,10,1,0,0,65.0,68.0,0.0,73.0,71.0,75.0,0,0.0,10,1,0,0,82.0,88.0,0.0,89.0,81.0,96.0,0,0.0,10,1,0,0,72.5,76.0,0.0,107.0,106.0,107.0,0,0.0,10,1,0,0,12.0,14.0,19.0,38.0,38.0,38.0,0,1.0,10,0,0,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,0,0,82.0,82.0,37.0,62.0,62.0,62.0,3,32.0,10,0,0,0,0
2,2000,3019,3,2000.0,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,3,44.0,10,0,0,0,24.5,24.5,49.0,27.0,27.0,27.0,3,48.0,10,0,0,0,160.0,165.0,0.0,198.0,192.0,204.0,0,0.0,10,1,0,0,65.0,68.0,0.0,73.0,71.0,75.0,0,0.0,10,1,0,0,82.0,88.0,0.0,89.0,81.0,96.0,0,0.0,10,1,0,0,72.5,76.0,0.0,107.0,106.0,107.0,0,0.0,10,1,0,0,12.0,14.0,19.0,38.0,38.0,38.0,0,1.0,10,0,0,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,0,0,82.0,82.0,37.0,62.0,62.0,62.0,3,32.0,10,0,0,0,0
3,2000,3019,4,2000.0,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,3,44.0,10,0,0,0,24.5,24.5,49.0,27.0,27.0,27.0,3,48.0,10,0,0,0,160.0,165.0,0.0,198.0,192.0,204.0,0,0.0,10,1,0,0,65.0,68.0,0.0,73.0,71.0,75.0,0,0.0,10,1,0,0,82.0,88.0,0.0,89.0,81.0,96.0,0,0.0,10,1,0,0,72.5,76.0,0.0,107.

In [102]:
main_df[['target_year','day']] = main_df[['target_year','day']].astype('uint16')

In [103]:
drop_columns_list = main_df.select_dtypes(include=np.number).columns[(main_df.select_dtypes(include=np.number).sum() == 0)]
main_df.drop(columns=drop_columns_list, inplace=True)

In [104]:
drop_columns_list = main_df.select_dtypes(include='O').columns[main_df.select_dtypes(include='O').describe().loc['unique'] == 1]
main_df.drop(columns=drop_columns_list, inplace=True)
cat_col = list(main_df.select_dtypes(include='O').columns)
main_df[cat_col] = main_df[cat_col].astype('uint16')
main_df[cat_col] = main_df[cat_col].astype('category')

In [105]:
norm_df = main_df

### Нормируем фичи

In [106]:
#from sklearn.preprocessing import StandardScaler

#ids, data, target = main_df[main_df.columns[:4]], main_df[main_df.columns[4:-1]], main_df[main_df.columns[-1]]

#scaler = StandardScaler()
#scaler.fit(data)
#transformed_data = scaler.transform(data)
#norm_df = pd.concat([ids, pd.DataFrame(transformed_data, columns = main_df.columns[4:-1]), target], axis=1)
#norm_df

In [107]:
X_train_col

[1985,
 1986,
 1987,
 1990,
 1991,
 1994,
 1995,
 1998,
 1999,
 2006,
 2007,
 2008,
 2009,
 2010,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [108]:
X_test_col

[2000, 2002, 2004, 2011, 2012, 1988, 1992, 1996, 2003]

In [112]:
# Разделим на трейн и тест исходя из target_year

test = norm_df[norm_df.target_year.isin(X_test_col)].reset_index(drop=True).dropna()
train = norm_df[norm_df.target_year.isin(X_train_col)].reset_index(drop=True).dropna()

# target_year больше не нужна

test.drop(columns=['target_year'], inplace=True)
train.drop(columns=['target_year'], inplace=True)

In [113]:
# Поделим данные на предикторы и таргет

X_train, y_train = train.iloc[:, :-1], train.ice_jam
X_test, y_test = test.iloc[:, :-1], test.ice_jam

In [114]:
X_test

,year,station_id,day,drainage_area,z_null,1_median_stage_min,1_median_stage_max,1_median_snow_height,1_first_stage_avg,1_first_stage_min,1_first_stage_max,1_first_snow_height,2_median_stage_min,2_median_stage_max,2_median_snow_height,2_first_stage_avg,2_first_stage_min,2_first_stage_max,2_first_snow_height,6_median_stage_min,6_median_stage_max,6_first_stage_avg,6_first_stage_min,6_first_stage_max,6_first_water_code,6_first_snow_coverage_station,6_first_snow_height_q1,7_median_stage_min,7_median_stage_max,7_first_stage_avg,7_first_stage_min,7_first_stage_max,7_first_snow_coverage_station,7_first_snow_height_q1,8_median_stage_min,8_median_stage_max,8_first_stage_avg,8_first_stage_min,8_first_stage_max,8_first_snow_coverage_station,8_first_snow_height_q1,9_median_stage_min,9_median_stage_max,9_first_stage_avg,9_first_stage_min,9_first_stage_max,9_first_snow_coverage_station,9_first_snow_height_q1,10_median_stage_min,10_median_stage_max,10_median_snow_height,10_first_stage_avg,10_first_stage_min,10_first_stage_max,10_first_snow_height,10_first_snow_coverage_station,10_first_snow_height_q1,11_median_stage_min,11_median_stage_max,11_median_snow_height,11_first_stage_avg,11_first_stage_min,11_first_stage_max,11_first_water_code,11_first_snow_height,11_first_snow_coverage_station,11_first_snow_height_q1,12_median_stage_min,12_median_stage_max,12_median_snow_height,12_first_stage_avg,12_first_stage_min,12_first_stage_max,12_first_snow_height
0,2000,3019,1,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,44.0,24.5,24.5,49.0,27.0,27.0,27.0,48.0,160.0,165.0,198.0,192.0,204.0,0,10,1,65.0,68.0,73.0,71.0,75.0,10,1,82.0,88.0,89.0,81.0,96.0,10,1,72.5,76.0,107.0,106.0,107.0,10,1,12.0,14.0,19.0,38.0,38.0,38.0,1.0,10,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,82.0,82.0,37.0,62.0,62.0,62.0,32.0
1,2000,3019,2,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,44.0,24.5,24.5,49.0,27.0,27.0,27.0,48.0,160.0,165.0,198.0,192.0,204.0,0,10,1,65.0,68.0,73.0,71.0,75.0,10,1,82.0,88.0,89.0,81.0,96.0,10,1,72.5,76.0,107.0,106.0,107.0,10,1,12.0,14.0,19.0,38.0,38.0,38.0,1.0,10,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,82.0,82.0,37.0,62.0,62.0,62.0,32.0
2,2000,3019,3,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,44.0,24.5,24.5,49.0,27.0,27.0,27.0,48.0,160.0,165.0,198.0,192.0,204.0,0,10,1,65.0,68.0,73.0,71.0,75.0,10,1,82.0,88.0,89.0,81.0,96.0,10,1,72.5,76.0,107.0,106.0,107.0,10,1,12.0,14.0,19.0,38.0,38.0,38.0,1.0,10,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,82.0,82.0,37.0,62.0,62.0,62.0,32.0
3,2000,3019,4,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,44.0,24.5,24.5,49.0,27.0,27.0,27.0,48.0,160.0,165.0,198.0,192.0,204.0,0,10,1,65.0,68.0,73.0,71.0,75.0,10,1,82.0,88.0,89.0,81.0,96.0,10,1,72.5,76.0,107.0,106.0,107.0,10,1,12.0,14.0,19.0,38.0,38.0,38.0,1.0,10,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,82.0,82.0,37.0,62.0,62.0,62.0,32.0
4,2000,3019,5,92200,249.38,39.0,39.0,47.0,74.0,74.0,74.0,44.0,24.5,24.5,49.0,27.0,27.0,27.0,48.0,160.0,165.0,198.0,192.0,204.0,0,10,1,65.0,68.0,73.0,71.0,75.0,10,1,82.0,88.0,89.0,81.0,96.0,10,1,72.5,76.0,107.0,106.0,107.0,10,1,12.0,14.0,19.0,38.0,38.0,38.0,1.0,10,0,-51.0,-51.0,26.0,-82.0,-82.0,-82.0,3,23.0,10,0,82.0,82.0,37.0,62.0,62.0,62.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,1996,3027,40,199000,175.51,72.0,72.0,55.0,101.0,101.0,101.0,47.0,62.5,62.5,55.0,65.0,65.0,65.0,54.0,702.0,707.0,632.0,614.0,650.0,0,10,1,413.0,424.0,510.0,502.0,518.0,10,1,111.0,115.0,286.0,283.0,289.0,10,1,137.0,138.5,139.0,139.0,139.0,10,1,42.0,43.0,25.0,95.0,93.0,97.0,12.0,10,0,-22.0,-21.5,36.0,-14.0,-14.0,-14.0,2,39.0,10,0,108.0,108.0,49.0,87.0,87.0,87.0,49.0
2460,1996,3027,41,199000,175.51,72.0,72.0,55.0,101.0,101.0,101.0,47.0,62.5,62.5,55.0,65.0,65.0,65.0,54.0,702.0,707.0,632.0,614.0,650

### CatBoostClassifier

In [115]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [116]:
category_columns = X_train.select_dtypes(include='category')

In [117]:
model = CatBoostClassifier(depth=2,
                           cat_features=category_columns,
                           loss_function='Logloss',
                           verbose=False)
parameters = {'iterations':range(50,170,20),'learning_rate':np.arange(0.1,0.2,0.02)}
GS = GridSearchCV(model, parameters, scoring='f1_macro')
# train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))
# make the prediction using the resulting model
#preds_class = model.predict(test_data)
#preds_proba = model.predict_proba(test_data)
#print("class = ", preds_class)
#print("proba = ", preds_proba)

0.49724546011018156


In [135]:
answ = pd.DataFrame(model.predict_proba(X_test), index=X_test.index)#.describe()
answ

,0,1
0,0.998879,0.001121
1,0.998879,0.001121
2,0.998879,0.001121
3,0.998879,0.001121
4,0.998879,0.001121
...,...,...
2459,0.999526,0.000474
2460,0.999526,0.000474
2461,0.999526,0.000474
2462,0.999526,0.000474


In [150]:
answ['ice_jam'] = answ[0].apply(lambda x: x < 0.988635)
answ['ice_jam'] = answ['ice_jam'].astype('uint8')
answ['ice_jam']

0       0
1       0
2       0
3       0
4       0
       ..
2459    0
2460    0
2461    0
2462    0
2463    0
Name: ice_jam, Length: 2464, dtype: uint8

In [154]:
test_df['ice_jam'] = answ['ice_jam']
test_df = test_df.fillna(0)
test_df['ice_jam'] = test_df['ice_jam'].astype('uint8')
test_df

,year,station_id,day,ice_jam
0,2001,3019,0,0
1,2001,3019,1,0
2,2001,3019,2,0
3,2001,3019,3,0
4,2001,3019,4,0
...,...,...,...,...
3901,1997,3027,39,0
3902,1997,3027,40,0
3903,1997,3027,41,0
3904,1997,3027,42,0


In [155]:
test_df['ice_jam'].value_counts()

0    3288
1     618
Name: ice_jam, dtype: int64

In [120]:
pd.Series(y_pred).value_counts()

0    2464
dtype: int64

In [123]:
test_task_1_random['ice_jam'] = 0
test_task_1_random

,year,station_id,day,ice_jam
0,2001,3019,0,0
1,2001,3019,1,0
2,2001,3019,2,0
3,2001,3019,3,0
4,2001,3019,4,0
...,...,...,...,...
3901,1997,3027,39,0
3902,1997,3027,40,0
3903,1997,3027,41,0
3904,1997,3027,42,0


In [156]:
test_df.to_csv(r'e:\Documents\mchs\submission.csv', index=True)

In [72]:
#X_test.station_id.unique()

In [73]:
#test_task_1_random.station_id.unique()

In [74]:
#test_task_1_random['ice_jam'] = y_pred

In [75]:
#X_test.year.value_counts().sort_index()

In [76]:
#test_task_1_random.year.value_counts().sort_index()